<a href="https://colab.research.google.com/github/justaraj/Ericsson/blob/main/ErricsonFinal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch

torch.cuda.empty_cache()

In [2]:
import torch
major_version, minor_version = torch.cuda.get_device_capability()
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
if major_version >= 8:
    !pip install --no-deps packaging ninja einops flash-attn xformers trl peft accelerate bitsandbytes
    !pip install triton # Install triton explicitly for newer GPUs
else:
    !pip install --no-deps xformers trl peft accelerate bitsandbytes
    !pip install triton  # Install triton explicitly for older GPUs
pass

  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-install-avvfrnm_/unsloth_635522595b754717a473f8b34785c0e6
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-install-avvfrnm_/unsloth_635522595b754717a473f8b34785c0e6
  Resolved https://github.com/unslothai/unsloth.git to commit d91d40a7b6b556f2d1fdd3e1e430f7a76a799627
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [3]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Choose any! Llama 3 is up to 8k
dtype = None
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

fourbit_models = [
    "unsloth/mistral-7b-bnb-4bit",
    "unsloth/mistral-7b-instruct-v0.2-bnb-4bit",
    "unsloth/llama-2-7b-bnb-4bit",
    "unsloth/gemma-7b-bnb-4bit",
    "unsloth/gemma-7b-it-bnb-4bit",
    "unsloth/gemma-2b-bnb-4bit",
    "unsloth/gemma-2b-it-bnb-4bit",
    "unsloth/llama-3-8b-bnb-4bit",
]

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/gemma-2b-bnb-4bit",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,

)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
==((====))==  Unsloth 2024.8: Fast Gemma patching. Transformers = 4.44.2.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.4.0+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.27.post2. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/2.07G [00:00<?, ?B/s]

`config.hidden_act` is ignored, you should use `config.hidden_activation` instead.
Gemma's activation function will be set to `gelu_pytorch_tanh`. Please, use
`config.hidden_activation` if you want to override this behaviour.
See https://github.com/huggingface/transformers/pull/29402 for more details.


generation_config.json:   0%|          | 0.00/154 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/40.0k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

In [4]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth 2024.8 patched 18 layers with 18 QKV layers, 18 O layers and 18 MLP layers.


In [5]:
# this is basically the system prompt
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

EOS_TOKEN = tokenizer.eos_token # do not forget this part!
def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    inputs       = examples["input"]
    outputs      = examples["output"]
    texts = []
    for instruction, input, output in zip(instructions, inputs, outputs):
        text = alpaca_prompt.format(instruction, input, output) + EOS_TOKEN # without this token generation goes on forever!
        texts.append(text)
    return { "text" : texts, }
pass

from datasets import load_dataset
dataset = load_dataset("yahma/alpaca-cleaned", split = "train")
dataset = dataset.map(formatting_prompts_func, batched = True,)

Generating train split:   0%|          | 0/51760 [00:00<?, ? examples/s]

Map:   0%|          | 0/51760 [00:00<?, ? examples/s]

In [6]:
import pandas as pd
from datasets import Dataset, load_dataset

# Load your custom dataset from the CSV file using pandas
df = load_dataset("Aakanksha26/train", split = "train")  # Ensure the correct path to your CSV file

# Convert pandas DataFrame to Hugging Face Dataset
dataset = Dataset.from_pandas(df.to_pandas())


Generating train split:   0%|          | 0/22 [00:00<?, ? examples/s]

In [7]:
from transformers import AutoTokenizer

# Define the tokenizer (replace 'model_name' with your specific model)
model_name = "unsloth/gemma-2b-bnb-4bit"
tokenizer = AutoTokenizer.from_pretrained("unsloth/gemma-2b-bnb-4bit")  # Example: "gpt2", "bert-base-uncased"
EOS_TOKEN = tokenizer.eos_token

# Alpaca prompt formatting template
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

# Function to format the dataset prompts
def formatting_prompts_func(examples):
    use_cases = examples["use_case"]
    details = examples[" details "]
    process_flows = examples["process_flow"]
    texts = []
    for use_case, detail, process_flow in zip(use_cases, details, process_flows):
        text = alpaca_prompt.format(use_case, detail, process_flow) + EOS_TOKEN
        texts.append(text)
    return {"text": texts}

# Apply the formatting function
formatted_dataset = dataset.map(formatting_prompts_func, batched=True)


tokenizer_config.json:   0%|          | 0.00/40.0k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

Map:   0%|          | 0/22 [00:00<?, ? examples/s]

In [8]:
# Check the first few rows of the formatted dataset
formatted_dataset.to_pandas().head()


,use_case,details,process_flow,text
0,General Reporting,None,None,"Below is an instruction that describes a task,..."
1,Voice CDR,System shall ingest Call Detail Records (CDR) ...,User uploads voice CDR files to the system.; S...,"Below is an instruction that describes a task,..."
2,Rejected CDR,System shall generate reports for rejected CDRs.,User accesses the Rejected CDR report section....,"Below is an instruction that describes a task,..."
3,Carrier Tarif,System shall manage carrier tariff data.; Syst...,User inputs carrier tariff data.; System store...,"Below is an instruction that describes a task,..."
4,Invoices,System shall generate invoices for services pr...,User initiates invoice generation.; System com...,"Below is an instruction that describes a task,..."


from matplotlib import pyplot as plt
import seaborn as sns
_df_0.groupby('use_case').size().plot(kind='barh', color=sns.palettes.mpl_palette('Dark2'))
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
import seaborn as sns
_df_1.groupby(' details ').size().plot(kind='barh', color=sns.palettes.mpl_palette('Dark2'))
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
import seaborn as sns
_df_2.groupby('process_flow').size().plot(kind='barh', color=sns.palettes.mpl_palette('Dark2'))
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
import seaborn as sns
_df_3.groupby('text').size().plot(kind='barh', color=sns.palettes.mpl_palette('Dark2'))
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
import seaborn as sns
import pandas as pd
plt.subplots(figsize=(8, 8))
df_2dhist = pd.DataFrame({
    x_label: grp[' details '].value_counts()
    for x_label, grp in _df_4.groupby('use_case')
})
sns.heatmap(df_2dhist, cmap='viridis')
plt.xlabel('use_case')
_ = plt.ylabel(' details ')

from matplotlib import pyplot as plt
import seaborn as sns
import pandas as pd
plt.subplots(figsize=(8, 8))
df_2dhist = pd.DataFrame({
    x_label: grp['process_flow'].value_counts()
    for x_label, grp in _df_5.groupby(' details ')
})
sns.heatmap(df_2dhist, cmap='viridis')
plt.xlabel(' details ')
_ = plt.ylabel('process_flow')

from matplotlib import pyplot as plt
import seaborn as sns
import pandas as pd
plt.subplots(figsize=(8, 8))
df_2dhist = pd.DataFrame({
    x_label: grp['text'].value_counts()
    for x_label, grp in _df_6.groupby('process_flow')
})
sns.heatmap(df_2dhist, cmap='viridis')
plt.xlabel('process_flow')
_ = plt.ylabel('text')

In [9]:
!pip install -q -U bitsandbytes peft transformers accelerate
from peft import prepare_model_for_kbit_training, LoraConfig, get_peft_model
import torch

# Prepare the quantized model for k-bit training
model = prepare_model_for_kbit_training(model)

# Define the Lora configuration
config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=["self_attn.q_proj", "self_attn.k_proj", "self_attn.v_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
)

# Create a Peft model with Lora
model = get_peft_model(model, config)
model.print_trainable_parameters()

from trl import SFTTrainer
from transformers import TrainingArguments

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = formatted_dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        max_steps = 240,
        learning_rate = 2e-4,
        fp16 = not torch.cuda.is_bf16_supported(),
        bf16 = torch.cuda.is_bf16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
    ),
)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 324.4/324.4 kB 23.0 MB/s eta 0:00:00
trainable params: 18,358,272 || all params: 3,048,818,688 || trainable%: 0.6021


Map (num_proc=2):   0%|          | 0/22 [00:00<?, ? examples/s]

max_steps is given, it will override any value given in num_train_epochs


In [12]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 22 | Num Epochs = 120
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 240
 "-____-"     Number of trainable parameters = 18,358,272


Step,Training Loss
1,0.038600
2,0.032200
3,0.042300
4,0.033400
5,0.038100
6,0.041300
7,0.038000
8,0.044400
9,0.034300
10,0.034600


In [11]:
# alpaca_prompt = Copied from above
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    alpaca_prompt.format(
        "The system shall handle the creation, processing, and storage of invoices. The system shall provide functionalities to generate invoices based on the data provided, manage invoice status, and store historical invoice data.", # instruction
        "Invoice Management", # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 64, use_cache = True)
tokenizer.batch_decode(outputs)

['<bos>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction:\nThe system shall handle the creation, processing, and storage of invoices. The system shall provide functionalities to generate invoices based on the data provided, manage invoice status, and store historical invoice data.\n\n### Input:\nInvoice Management\n\n### Response:\nUser creates invoices.; System stores and processes the invoice.; System generates and sends the invoice to the customer.<eos>']

In [13]:
# alpaca_prompt = Copied from above
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    alpaca_prompt.format(
        "The system shall handle the creation, processing, and storage of invoices. The system shall provide functionalities to generate invoices based on the data provided, manage invoice status, and store historical invoice data.", # instruction
        "Invoice Management", # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)

<bos>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
The system shall handle the creation, processing, and storage of invoices. The system shall provide functionalities to generate invoices based on the data provided, manage invoice status, and store historical invoice data.

### Input:
Invoice Management

### Response:
User creates invoices.; System stores and processes the invoice.; System generates and sends the invoice to the customer.<eos>


In [14]:
# alpaca_prompt = Copied from above
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    alpaca_prompt.format(
        "System shall generate volume commitment reports.", # instruction
        "Volume Commitment Reports", # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)

<bos>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
System shall generate volume commitment reports.

### Input:
Volume Commitment Reports

### Response:
User accesses the Volume Commitment Reports section.; User specifies the criteria for the report.; System retrieves data and generates the report.<eos>
